[AgiBot World Colosseo: Large-scale Manipulation Platform for Scalable and Intelligent Embodied Systems](https://agibot-world.com/blog/go1)

# AGIBOTWORLD: MODEL
<!-- To effectively utilize our high-quality AgiBot World dataset and enhance the policy’s generalizability, we propose a hierarchical Vision-Language-Latent-Action (ViLLA) framework with three training stages, as depicted in Fig. 4. Compared to Vision-Language-Action (VLA) model where action is vision-language conditioned, the ViLLA model predicts latent action tokens, conditioned on the generation of subsequent robot control actions. -->

为了有效利用我们高质量的 AgiBot World 数据集并增强策略的通用性, 我们提出了一个具有三个训练阶段的分层 Vision-Language-Latent-Action (ViLLA) 框架, 如图 4 所示。与视觉-语言-动作 (VLA) 模型(其中动作以视觉语言为条件)相比, ViLLA 模型以后续机器人控制动作的生成为条件来预测 latent action tokens。

<!-- In Stage 1, we project consecutive images into a latent action space by training an encoder-decoder latent action model (LAM) on internet-scale heterogeneous data. This allows the latent action to serve as an intermediate representation, bridging the gap between general image-text inputs and robotic actions. In Stage 2, these latent actions act as pseudo-labels for the latent planner, facilitating embodiment-agnostic long-horizon planning and leveraging the generalizability of the pre-trained VLM. Finally, in Stage 3, we introduce the action expert and jointly train it with the latent planner to support the learning of dexterous manipulation. -->

在第 1 阶段, 我们通过在互联网规模的异构数据上训练编码器-解码器 latent action model (LAM), 将连续图像投射到 latent action 空间。这使得 latent action 可以作为中间表示, 弥合通用图像文本输入和机器人动作之间的差距。在第 2 阶段, 这些 latent actions 充当 latent 规划器的伪标签, 便于与具身无关的长期规划并利用预训练 VLM 的通用性。最后, 在第 3 阶段, 我们引入动作专家并将其与 latent 规划器联合训练, 以支持灵巧操作的学习。

## Latent Action Model
<!-- Despite considerable advancements in gathering diverse robot demonstrations, the volume of action-labeled robot data remains limited relative to web-scale datasets. To broaden the data pool by incorporating internet-scale human videos lacking action labels and cross-embodiment robot data, we employ latent actions [30] in Stage 1 to model the inverse dynamics of consecutive frames. This approach enables the transfer of real-world dynamics from heterogeneous data sources into universal manipulation knowledge. -->

尽管在收集多样化的机器人演示方面取得了显著进展, 但相对于互联网规模的数据集, 带有动作标签的机器人数据量仍然有限。为了通过整合缺乏动作标签的互联网规模的人类视频和跨实体机器人数据来扩大数据池, 我们在第 1 阶段采用 latent actions [30] 来建模连续帧的逆动力学。这种方法可以将现实世界的动态从异构数据源转移到通用操纵知识中。

<!-- To extract latent actions from video frames $\{I_t,I_{t+H} \}$, the latent action model is constructed around an inverse dynamics model-based encoder $\mathbf{I}(z_t|I_t,I_{t+H})$ and a forward dynamics model-based decoder $\mathbf{F}(I_{t+H}|I_t,z_t)$. The encoder employs a spatial-temporal transformer [31] with casual temporal masks following Bruce et al. [30], while the decoder is a spatial transformer that takes the initial frame and discretized latent action tokens $z_t = [z^0_t, \dots ,z^{k−1}_t]$ as input, with $k$ set to 4. The latent action tokens are quantized using a VQ-VAE objective [32], with a codebook of size $|C|$. -->

为了从视频帧 $\{I_t,I_{t+H} \}$ 中提取 latent actions, latent action 模型由一个基于逆动力学模型的编码器 $\mathbf{I}(z_t|I_t,I_{t+H})$ 和一个基于前向动力学模型的解码器 $\mathbf{F}(I_{t+H}|I_t,z_t)$ 构建。编码器采用时空 transformer [31], 并遵循 Bruce et al. [30] 的因果时间掩码, 而解码器是一个空间 transformer, 以初始帧和离散的 latent action tokens $z_t = [z^0_t, \dots ,z^{k−1}_t]$ 作为输入, $k$ 设置为 4。使用 VQ-VAE 目标 [32] 量化 latent action tokens, 代码本大小为 $|C|$。
                                   
## Latent Planner
<!-- With the aim of establishing a solid foundation for scene and object understanding and general reasoning ability, the ViLLA model harnesses a VLM pre-trained on web-scale vision-language data and incorporates a latent planner for embodiment-agnostic planning within the latent action space. We use InternVL2.5-2B [33] as the VLM backbone due to its strong transfer learning capabilities. The two-billion parameter scale has proven effective for robotic tasks in our preliminary experiments, as well as in prior studies [10], [26]. Multiview image observations are first encoded using InternViT before being projected into the language space. The latent planner consists of 24 transformer layers, which enable layer-by-layer conditioning from the VLM backbone with full bidirectional attention. -->

为了奠定场景和物体理解以及一般推理能力的坚实基础, ViLLA 模型利用在互联网规模的视觉语言数据上预训练的 VLM, 并耦合一个 latent 规划器, 用于 latent action 空间上的具身无关的规划。我们使用 InternVL2.5-2B [33] 作为 VLM 骨干, 因为其强大的迁移学习能力。在我们的初步实验以及先前的研究 [10]、[26] 中, 20 亿参数规模已被证明对机器人任务是有效的。首先使用 InternViT 对多视角图像观测进行编码, 然后将其投影到语言空间中。latent 规划器由 24 个 transformer 层组成, 它们能够从 VLM 骨干进行逐层调节, 并具有完全双向注意力。

<!-- Specifically, given multiview input images $(I^h_t, I^l_t, I^r_t)$ (typically from the head, left wrist, and right wrist) at timestep $t$, along with a language instruction $l$ describing the ongoing task, the latent planner predicts latent action tokens: $\mathbf{P}(z_t|I^h_t,I^l_t,I^r_t,l)$, with supervision produced by the LAM encoder based on the head view: $z_t := \mathbf{I}(I^h_t,I^h_{t+H})$. Since the latent action space is orders of magnitude smaller than the discretized low-level actions used in OpenVLA [4], this approach also facilitates the efficient adaptation of general-purpose VLMs into robot policies. -->

具体而言, 给定时间步长 $t$ 的多视角输入图像 $(I^h_t, I^l_t, I^r_t)$ (通常来自头部、左手腕和右手腕), 以及描述当前任务的语言指令 $l$, latent 规划器预测 latent action tokens: $\mathbf{P}(z_t|I^h_t,I^l_t,I^r_t,l)$，其中 LAM 编码器基于头部视角生成监督: $z_t := \mathbf{I}(I^h_t,I^h_{t+H})$。由于 latent action 空间比 OpenVLA [4] 中使用的离散化下层动作小几个数量级, 这种方法也有助于将通用 VLM 高效适配到机器人策略。

## Action Expert
<!-- To achieve high-frequency and dexterous manipulation, Stage 3 integrates an action expert that utilizes a diffusion objective to model the continuous distribution of low-level actions [34]. Although the action expert shares the same architectural framework as the latent planner, their objectives diverge: the latent planner generates discretized latent action tokens through masked language modeling, while the action expert regresses low-level actions via an iterative denoising process. Both expert modules are conditioned hierarchically on preceding modules, including the action expert itself, ensuring coherent integration and information flow within the dual-expert system. -->

为了实现高频和灵巧的操作, 第 3 阶段集成一个动作专家, 该专家利用扩散目标来建模下层动作的连续分布 [34]。尽管动作专家与 latent 规划器共享相同的架构框架, 但它们的目标有所不同: latent 规划器通过掩码语言建模生成离散化的 latent action tokens, 而动作专家通过迭代去噪过程回归下层动作。这两个专家模块在层次上都以先前的模块(包括动作专家自身)为条件, 从而确保双专家系统内的连贯集成和信息流。

The action expert decodes low-level action chunks, de-noted by $A_t= [a_t,a_{t+1}, \dots,a_{t+H}]$ with $H=30$, using proprioceptive state $p_t$ over an interval of $H$ timesteps: $\mathbf{A}(A_t|I^h_t,I^l_t,I^r_t,p_t,l)$. During inference, the VLM, latent planner, and action expert are synergistically combined within the generalist policy GO-1, which initially predicts $k$ latent action tokens and subsequently conditions the denoising process to produce the final control signals.

动作专家解码低级动作块，表示为 $A_t= [a_t,a_{t+1}, \dots,a_{t+H}]$，其中 $H=30$，使用本体感受状态 $p_t$，时间间隔为 $H$ 个时间步长：$\mathbf{A}(A_t|I^h_t,I^l_t,I^r_t,p_t,l)$。在推理过程中，VLM、潜在规划器和动作专家在通用策略 GO-1 中协同结合，该策略最初预测 $k$ 个潜在动作标记，随后调节去噪过程以产生最终控制信号。

动作专家使用本体状态 $p_t$ 在 $H$ 个时间步长的间隔内解码低级动作块，记为 $A_t= [a_t,a_{t+1}, \dots,a_{t+H}]$，其中 $H=30$：$\mathbf{A}(A_t|I^h_t,I^l_t,I^r_t,p_t,l)$。在推理过程中，VLM、潜在规划器和动作专家在通用策略 GO-1 中协同结合，GO-1 首先预测 $k$ 个潜在动作标记，然后条件化去噪过程以生成最终的控制信号。

In [1]:
from pypdf import PdfReader

pdf_path = "/mnt/y/paper/AgiBot_GO-1.pdf"

reader = PdfReader(pdf_path)
number_of_pages = len(reader.pages)
page = reader.pages[5]
text = page.extract_text()
print(text)

LargeVision-languageModel…InternViTTokenizerLatent PlannerAction Expert
Task InstructionsMultiviewImagesLAMEncoderLAMDecoderLatentAction Tokens
𝐼!"#𝐼!𝑧$𝑧%𝑧&𝑧'(%
𝐼#!"#
“Hang the T-shirt”𝑎$𝑎%𝑎&𝑎#(%ActionChunk
…Latent ActionRobotAction…Reconstructed
HumanVideos
Web-scaleVision-languageData
Cross-EmbodimentRobotDataGO-1𝑝𝑎)𝑧$𝑧%𝑧&𝑧'(%Latent PlanningStage-1
Stage-2Stage-3Image& Text
AgiBotWorld
……𝑧)…𝑎*…Pre-trainingData
𝑧)Fig. 4: We propose GO-1 , a generalist policy featuring general reasoning and long-horizon planning capabilities. The latent
action model (LAM) learns universal action representations from web-scale video data ( i.e.,human videos from Ego4D),
and quantizes them into discrete latent action tokens. The latent planner conducts temporal reasoning through latent action
prediction, bridging the gap between image-text inputs and robot actions generated by the action expert.
vision-language data and incorporates a latent planner for
embodiment-agnostic planning within the latent acti

/home/cfz/miniconda3/envs/learning/lib/python3.10/site-packages/pypdf/_crypt_providers/_cryptography.py:32: CryptographyDeprecationWarning: ARC4 has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.ARC4 and will be removed from this module in 48.0.0.
  from cryptography.hazmat.primitives.ciphers.algorithms import AES, ARC4
